In [1]:
import numpy as np
import random
import gurobipy as gp
import time

Variables to be used to generate nodes and cluster

In [ ]:
np.random.seed(42)

# Number of nodes
n = 200
# Number of clusters
p = 2

capacity_mean = 10
capacity_stddev = 2
weight_mean = 1
weight_stddev = 0.1
lambda_param = 0.5

Generate Random Instances using capacity mean and standard deviation

In [2]:
def generate_instances(n, p, capacity_mean, capacity_stddev):
    # Generate 2D positions for n nodes
    nodes = np.random.rand(n, 2)

    # Generate capacities for clusters
    capacities = np.random.normal(capacity_mean, capacity_stddev, p)

    return nodes, capacities

Generate Random Weights using weight mean and standard deviation

In [3]:
def generate_weights(n, weight_mean, weight_stddev):
    # Generate weights for n nodes
    weights = np.random.normal(weight_mean, weight_stddev, n)

    return weights

 Euclidean distance between two points p and q

In [4]:
def distance(p, q):
    return np.sqrt(np.sum((p-q)**2))

In [5]:
def solve_ccp(nodes, capacities, weights, lambda_param):
    n = nodes.shape[0]
    p = len(capacities)

    # Create gurobi model
    model = gp.Model('ccp')

    # Create decision variables
    x = {}
    y = {}

    # Update decision variables as mentioned in the problem statement
    for i in range(n):
        for j in range(p):
            x[i, j] = model.addVar(vtype=gp.GRB.BINARY, name=f'x[{i},{j}]')
        y[i] = model.addVar(vtype=gp.GRB.BINARY, name=f'y[{i}]')

    # Set objective function
    obj = gp.quicksum(distance(nodes[i], nodes[j]) * x[i,j] for i in range(n) for j in range(p))
    obj += lambda_param * gp.quicksum(capacities[j]*y[j] for j in range(p))
    obj -= lambda_param * gp.quicksum(weights[i]*x[i,j] for i in range(n) for j in range(p))
    model.setObjective(obj, gp.GRB.MINIMIZE)

    # Add constraints
    for i in range(n):
        model.addConstr(gp.quicksum(x[i,j] for j in range(p)) == 1, name=f'assign[{i}]')

    model.addConstr(gp.quicksum(y[j] for j in range(p)) <= p, name='num_clusters')

    for i in range(n):
        for j in range(p):
            model.addConstr(x[i,j] <= y[j], name=f'x_c[{i},{j}]')

    # Solve model
    model.optimize()
    # Extract solution
    clusters = []
    for j in range(p):
        cluster = [i for i in range(n) if x[i,j].X > 0.5]
        clusters.append(cluster)

    return clusters, model

In [6]:
def jaccard_similarity(clusters_1, clusters_2):
    jaccard_scores = []
    for cluster_1 in clusters_1:
        jaccard_scores_per_cluster = []
        for cluster_2 in clusters_2:
            union = len(set(cluster_1).union(cluster_2))
            intersection = len(set(cluster_1).intersection(cluster_2))
            jaccard_index = intersection / union
            jaccard_scores_per_cluster.append(jaccard_index)
        jaccard_scores.append(max(jaccard_scores_per_cluster))
    return np.mean(jaccard_scores)

In [7]:
def rand_index(clusters_1, clusters_2):
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    n = len(clusters_1)
    for i in range(n):
        for j in range(i+1, n):
            if (clusters_1[i] == clusters_1[j] and clusters_2[i] == clusters_2[j]):
                tp += 1
            elif (clusters_1[i] != clusters_1[j] and clusters_2[i] != clusters_2[j]):
                tn += 1
            elif (clusters_1[i] == clusters_1[j] and clusters_2[i] != clusters_2[j]):
                fp += 1
            else:
                fn += 1

    return (tp + tn) / (tp + tn + fp + fn)


In [ ]:
def insample_stability(clusters_reference,nodes, capacities, weights, lambda_param, num_runs=100):
    jaccard_scores = []
    for i in range(num_runs):
        clusters, _ = solve_ccp(nodes, capacities, weights, lambda_param)
        jaccard_scores.append(jaccard_similarity(clusters, clusters_reference))
    return np.mean(jaccard_scores)


In [8]:
def evaluate_out_of_sample_stability(clusters_reference, nodes, capacities, weights, lambda_param, num_trials=100):
    jaccard_scores = []
    for i in range(num_trials):
        # Generate new instances
        new_nodes, new_capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
        new_weights = generate_weights(n, weight_mean, weight_stddev)
        # Solve new instances
        new_clusters = solve_ccp(new_nodes, new_capacities, new_weights, lambda_param)
        
        # Evaluate Jaccard score
        jaccard_scores.append(jaccard_similarity(new_clusters, new_clusters))

    return np.mean(jaccard_scores)

In [13]:
def split_data(nodes, weights, train_ratio=0.7):
    n = nodes.shape[0]
    indices = np.arange(n)
    np.random.shuffle(indices)
    train_indices = indices[:int(n * train_ratio)]
    val_indices = indices[int(n * train_ratio):]
    return nodes[train_indices], weights[train_indices], nodes[val_indices], weights[val_indices]

def out_sample_stability(nodes, capacities, weights, lambda_param, num_splits=10):
    jaccard_scores = []
    for i in range(num_splits):
        nodes_train, weights_train, nodes_val, weights_val = split_data(nodes, weights)
        clusters_train,_ = solve_ccp(nodes_train, capacities, weights_train, lambda_param)
        clusters_val, _ = solve_ccp(nodes_val, capacities, weights_val, lambda_param)
        jaccard_scores.append(jaccard_similarity(clusters_val, clusters_train))
    return np.mean(jaccard_scores)

# Example usage
np.random.seed(42)

n = 200
p = 2
capacity_mean = 10
capacity_stddev = 2
weight_mean = 1
weight_stddev = 0.1
lambda_param = 0.5

nodes, capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
weights = generate_weights(n, weight_mean, weight_stddev)

out_sample_stability_score = out_sample_stability(nodes, capacities, weights, lambda_param)
print("DAKSFJHDLKSJF",out_sample_stability_score)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 421 rows, 420 columns and 842 nonzeros
Model fingerprint: 0xce7aba2e
Variable types: 0 continuous, 420 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-05, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 18.0858406
Presolve removed 421 rows and 420 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: -9.74717 18.0858 
No other solutions better than -9.74717

Optimal solution found (tolerance 1.00e-04)
Best objective -9.747168809116e+00, best bound -9.747168809116e+00, gap 0.0000%
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

In [ ]:
def split_data(nodes, weights, train_ratio=0.7):
    n = nodes.shape[0]
    indices = np.arange(n)
    np.random.shuffle(indices)
    train_indices = indices[:int(n * train_ratio)]
    val_indices = indices[int(n * train_ratio):]
    return nodes[train_indices], weights[train_indices], nodes[val_indices], weights[val_indices]


In [ ]:
def out_sample_stability(num_runs=10):
    rand_index_scores = []
    
    for i in range(num_runs):

        nodes, capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
        weights = generate_weights(n, weight_mean, weight_stddev)
        
        nodes_train, weights_train, nodes_val, weights_val = split_data(nodes, weights)
        
        clusters_train,_ = solve_ccp(nodes_train, capacities, weights_train, lambda_param)
        clusters_val,_ = solve_ccp(nodes_val, capacities, weights_val, lambda_param)
        rand_index_scores.append(rand_index(clusters_train, clusters_val))
    return np.mean(rand_index_scores)

In [ ]:
# Generate 2D nodes and random capacities for clusters
nodes, capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
# Generate random weight for each node
weights = generate_weights(n, weight_mean, weight_stddev)

clusters, model = solve_ccp(nodes, capacities, weights, lambda_param)

In [ ]:
in_sample_stability_score = insample_stability(clusters,nodes, capacities, weights, lambda_param, num_runs=100)
print("Insample Stability:",in_sample_stability_score)

In [ ]:

out_sample_stability_score = out_sample_stability(num_runs=100)
print("Outsample Stability:",out_sample_stability_score)